In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("EDA-Recommender") \
    .config("spark.driver.memory", "12g") \
    .getOrCreate()

/opt/conda/envs/recommender-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Genera el Profile Report

In [2]:
# ratings_pd = pd.read_csv("../data/raw_data/ratings.csv")

In [3]:
# profile = ProfileReport(ratings_pd, title="Ratings Profiling Report", explorative=True)
# profile.to_file("../data/ProfileReport/ratings_report.html")

In [4]:
# ratings = spark.createDataFrame(ratings_pd)

In [5]:
# movies_clean = spark.read \
#     .option("header", True) \
#     .option("encoding", "UTF-8") \
#     .csv("../data/clean_data/movies_clean")

In [6]:
df = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .option("multiLine", False) \
    .option("escape", "\"") \
    .csv("../data/raw_data/ratings.csv")
df.write.mode("overwrite").parquet("../data/raw_data/ratings")

In [7]:
ratings = spark.read.parquet("../data/raw_data/ratings")

In [8]:
movies_clean = spark.read.parquet("../data/clean_data/movies_clean")

Limpieza de datos

In [9]:
movies_clean.count()

40297

In [10]:
movies_clean.select("id").count()

40297

In [11]:
ratings.count()

26024289

In [12]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [13]:
ratings.select("movieId").distinct().count()

45115

In [14]:
ratings_clean = ratings.join(
    movies_clean.select("id"),
    ratings.movieId == movies_clean.id,
    "inner"
).drop(movies_clean.id)

In [15]:
ratings_clean.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [16]:
ratings_clean.select("movieId").distinct().count()

39676

In [17]:
ratings_clean.count()

23773869

In [18]:
ratings_clean.write \
    .mode("overwrite") \
    .parquet("../data/clean_data/ratings_clean")

In [ ]:
# ratingsc_pd = ratings_clean.toPandas()

In [ ]:
# profilec = ProfileReport(ratingsc_pd, title="Ratings Profiling Report", explorative=True)
# profilec.to_file("../data/ProfileReport/ratings_clean_report.html")

In [19]:
spark.stop()